# Basic stats on the evolution of COVID-19 hospitalizations in departments

In [1]:
%load_ext lab_black
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
from datetime import timedelta, date
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
pd.plotting.register_matplotlib_converters()

In [3]:
covid_url = (
    "https://www.data.gouv.fr/fr/datasets/r/63352e38-d353-4b54-bfd1-f1b3ee1cabd7"
)
filter_dep = ["971", "972", "973", "974", "976"]  # only metropolitan France
figsize = (15, 10)

#### <br> Load table with COVID-19 data (figures by dates)

In [4]:
df = pd.read_csv(covid_url, sep=";", parse_dates=True, index_col=2)
df = df.query("sexe == 0")  # sum male/female
df.drop(columns=["sexe"], inplace=True)
df.query("dep not in @filter_dep", inplace=True)
df.sort_index(inplace=True)
df.head(2)

,dep,hosp,rea,rad,dc
jour,,,,,
2020-03-18,01,2,0,1,0
2020-03-18,02,41,10,18,11


In [5]:
last_day = df.index.max()
previous_day = last_day - timedelta(days=1)
print("Evolution from ", previous_day, " to ", last_day)

Evolution from  2020-04-12 00:00:00  to  2020-04-13 00:00:00


#### <br> Build a new dataframe with data from two latest days and the associated evolution

In [6]:
sdf = df.loc[last_day]
sdf.reset_index(inplace=True)
pdf = df.loc[previous_day]
pdf.reset_index(inplace=True)
frame = {"dep": sdf["dep"], "prev_hosp": pdf["hosp"], "last_hosp": sdf["hosp"]}
stats = pd.DataFrame(frame)
stats["diff"] = stats["last_hosp"] - stats["prev_hosp"]
stats["percent"] = stats["diff"] / stats["prev_hosp"] * 100
stats.head()

,dep,prev_hosp,last_hosp,diff,percent
0,01,118,120,2,1.694915
1,02,283,286,3,1.060071
2,03,58,58,0,0.000000
3,04,34,33,-1,-2.941176
4,05,49,49,0,0.000000


#### <br> Departments with highest number of hospitalizations on the latest day

In [7]:
stats.sort_values(by="last_hosp", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
75,75,3249,3266,17,0.523238
94,94,2231,2249,18,0.806813
92,92,2024,2005,-19,-0.938735
93,93,1638,1619,-19,-1.159951
69,69,1241,1252,11,0.886382


#### <br> Min, max, mean of evolution between the two latest days (in sum and %)

In [8]:
print(
    "Min: ",
    stats["diff"].min(),
    " Mean: ",
    stats["diff"].mean(),
    " Max: ",
    stats["diff"].max(),
)

Min:  -19  Mean:  2.9479166666666665  Max:  21


In [9]:
print(
    "Min %: ",
    stats.percent.min(),
    " Mean %: ",
    stats.percent.mean(),
    " Max %: ",
    stats.percent.max(),
)

Min %:  -7.59493670886076  Mean %:  1.646947195988932  Max %:  19.230769230769234


#### <br> List of departments with a drecrease of hospitalizations (last two days)

In [10]:
stats.query("diff < 0")

,dep,prev_hosp,last_hosp,diff,percent
3,04,34,33,-1,-2.941176
13,14,162,156,-6,-3.703704
16,17,86,85,-1,-1.162791
19,21,344,340,-4,-1.162791
23,25,213,212,-1,-0.469484
24,26,236,234,-2,-0.847458
25,27,94,92,-2,-2.127660
27,29,123,121,-2,-1.626016
30,30,95,92,-3,-3.157895
31,31,244,243,-1,-0.409836


In [11]:
print(stats.query("diff < 0").shape)

(26, 5)


#### <br> Departments with highest evolution of hospitalizations (total number)

In [12]:
stats.sort_values(by="diff", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
9,10,239,260,21,8.786611
59,59,950,970,20,2.105263
94,94,2231,2249,18,0.806813
75,75,3249,3266,17,0.523238
77,77,811,828,17,2.096178


#### <br> Departments with highest rate of hospitalizations (%)

In [13]:
stats.sort_values(by="percent", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
22,24,26,31,5,19.230769
36,36,82,94,12,14.634146
10,11,81,91,10,12.345679
32,32,28,31,3,10.714286
9,10,239,260,21,8.786611


#### <br> Focus on some departments

In [14]:
stats.query("dep=='75'")

,dep,prev_hosp,last_hosp,diff,percent
75,75,3249,3266,17,0.523238


In [15]:
stats.query("dep=='71'")

,dep,prev_hosp,last_hosp,diff,percent
71,71,175,176,1,0.571429


In [16]:
stats.query("dep=='31'")

,dep,prev_hosp,last_hosp,diff,percent
31,31,244,243,-1,-0.409836
